# Introduction
 

 Taking derivatives is an ubiquitous tool that covers a wide range of applications - from mathematics, to physics and computer science, etc. 
 
 In machine learning, differentiation is extensively applied to the optimization problem by performing the gradient descent to minimize the cost function. 
Using a fast and versatile differentiation method, people can also get the second order derivatives and hessian matrices.
 
 Differentiation is also applied to solve everyday problems. For example, knowing the initial decomposition level of a contaminated food, 
 we can define a differential equation to predict how food will decay in the future and set an appropriate expiration date. 
 
 However, traditional methods of differentiation such as symbolic differentiation and numerical differentiation might be computationally expensive and less robust. Also, these methods do not scale well to vector functions with multiple variable inputs, which are widely used to solve real world problems.

 In this package, SmartDiff is a software package which implements a forward mode Automatic Differentiation (AD) package with chain rule to help users take derivatives easily. It automatically breaks the complex functions into small independent elementary operations based parentheses and operators. Then it calculates the derivatives in the small chunks step by step, moving outward to the entire function, to give the result. Therefore, SmartDiff is widely **generalizable** to different functions and **consumes very little memory**. It also computes the derivative of different components of the function in parallel, further speeding up the computation. 


# Background

## Chain Rule
 This automatic differentiation (AD) is performed mainly based on the chain rule. 
 The basic idea is to split the function into hierarchies of chunks containing the input variables and the built-in elementary operations in SmartDiff. 
 Therefore, from the innermost chunk(s), the calculation can be expressed in a series of steps iteratively, in which SmartDiff only considers one elementary operation at a time. 
 After the evaluation of both the function and its derivatives, the iteration can be traced in the computation graph. 

 Based on the key definition of chain rule, and known that $F(x) = f(g(x))$, we can represent it in multiple steps as follows:
    $\frac{\partial}{\partial x}(F(x)) =\frac{\partial}{\partial x}f(g(x))=f'(g(x))g'(x)$

 From the equation above, as long as we can compute the derivative of $f(x)$ and $g(x)$, we can get the derivative of $F(x)$.

Also, with such forward method, SmartDiff is able to evaluate each step of the calculation and keep track of function values and derivatives at each step, which makes the calculation faster and more robust.

## Graph Structure of Calculation
 SmartDiff is implemented based on a graph structure. For each node on the graph, it contains a variable or the result following one elementary operation. 
 For example, given a multivariable function $f(x,y) = (x^2+1)+\exp{(y)}$. To evaluate the function at $(x,y) = (1,1)$, we can write the evaluation trace as:

|     `Trace`        |   ` Operation`     |        `Value`         |    ` Derivative `       |  `Dev to x `|            `Dev to y`            |
|: -----------------:|:------------------:|:----------------------:|:-----------------------:|:-----------:|:--------------------------------:|
|        $x_1$       |       $x$          |          1             |       $ \dot{x}$        |       1     |                 0                |
|        $x_2$       |       $y$          |          1             |       $ \dot{y}$        |       0     |                 1                |
|        $v_1$       |     $x_1^2$        |          1             |       $2x_1\dot{x_1}$   |       2     |                 0                |
|        $v_2$       |     $v_1+1$        |          2             |          $\dot{v_1}$    |       2     |                 0                |
|        $v_3$       |     $\exp{(x_2)}$  |          $e$           |  $\exp{(x_2)}\dot{x_2}$ |       0     |               $e$                |
|         $f$        |     $v_2+v_3$      |          $e+2$         |  $\dot{v_2}+\dot{v_3}$  |       2     |               $e$                |

The traces above split the function into chunks of elementary operations and can be easily put into a graph structure, which is helpful for the differentiation calculation.

## Elementary Functions
 SmartDiff is mainly a combination of elementary operations in calculus, which contains:
 
 1. Arithmetic symbols (+, -, ×, /), parentheses, constant functions ($2$, $\pi$, $e$, etc.)
 2. Logarithmic functions, exponential functions, trigonometric functions, square root function, hyperbolic functions, etc. 

# How to use SmartDiff

1. How do you envision that a user will interact with your package? 

    The user first installs SmartDiff and calls the main script in the commandline and interacts with the Graphic User Interface (GUI). There is no need to import or instantiate python objects.
    
```python
    pip install SmartDiff
    python /dir/to/SmartDiff/main.py
```
 

# SmartDiff Graphic User Interface (GUI)

### Procedure:

Once SmartDiff is installed, it outputs the derivative(s) in the following steps.  

   1. The user chooses the dimension of the vector function from a drop down list (1,2,3) and the number of one-dimensional real-valued variables from a drop down list (1,2,3)
  
   2. The user inputs a specific data point (required)

   3. The user inputs a vector function by each component of the vector function

   4. The user checks the confirmation message, showing the function and the variable values.

   5. The user can choose to see the function value and the trace table in addition to the differentiation result.
  
   6. The user clicks Ok to differentiate.

   7. The program shows the results based on user specifications

### Advantages:

Using the buttons of functions, operators, and digits, the GUI can directly translate the user input into a defined string for latex rendering and a python expression for function evaluation and differentiation. This also alleviates the need to check for typos and undefined functions within the user input.


# Software Organization

### 1. Directory structure:
The following is the structure of the project after users download it from our github repository. 
The package we are developing is SmartDiff and the few files outside of it would be used in its packaging and distribution.
```python
    cs107-FinalProject/
        setup.py
        requirement.txt
        README.md
        LICENSE
        docs/
            milestone1.ipynb
            
        SmartDiff/
            __init__.py
            main.py  # start up script
            preprocess/  # This is a (sub)package
                __init__.py 
                parse_gui_input.py   # A GUI input parser module
            solvers/ #  calculate derivatives and function values at the given point
                __init__.py
                element_op.py  # define the derivatives of elementary operations
                integrator.py  # go through the preprocessed python expression and call the corresponding elementary operations
                find_value.py  # calculate the function value based on preprocessed output
                trace_table/   # generate the trace table during forward AD
                    __init__.py
                    gen_table.py # generate the table based on integrator.py output
                    visualize_table.py # generate a .png file of the trace table
            GUI/  # create a PyQt5-based GUI that takes in user input and displays computation output
                __init__.py
                UI_main.py  # coordinate between different UI submodules
                
                layout/  # configure the window layout for each step
                    __init__.py
                    step1.py
                    step2.py
                    step3.py
                    step4.py
                    step5.py
                    step6.py
                    step7.py
                    all.py  # the window configuration shared by all steps
                    shared.py  # the window configuration shared by step 2 and 3

                config_layout/  # Changes the window based on user input in the previous step
                    __init__.py
                    confit.py
                
                msg/  # generate instructions and error message (sometimes based on other modules) for the user (all steps)
                    __init__.py
                    error_msg.py
                    instruction.py
                
            tests/ # extensive tests of each module and boundaries between different modules
                __init__.py
                preprocess_test.py
                solvers_test.py
                UI_test.py
                UI_preprocess_test.py
                preprocess_solvers_test.py
                solvers_UI.py
```

### 2. Testing Procedure
For each function, we will have some basic test cases in the docstring. For extensive testing between the boundaries of different modules, we will implement several test functions for each module 
on separate files under /tests. These tests will include the functions *element_op_tests*, *integrator_tests*, *find_value_tests*, and *trace_table_tests*, which will check the correct definition of 
derivatives for elementary operations, the successful implementation of those derivatives in a given equation, and the correct generation of a trace table, respectively. All of the tests functions will
be called under a main test calling function, *all_tests*, to ensure that all the tests are running. The automatic testing of our code will be conducted using TravisCI and the proportion of code 
tested will be monitored by CodeCov.

### 3. Packaging and Package Distribution
We will be using Python Package Index (PyPI) to pack up our project and distribute it to the Python user community. 
The PyPI of our package is "smartdiff", the same name to be used in pip install.  
setup.py file is a build script of setuptools and will be used to generate distribution archive files. Then we generate our project API token and upload the files to TestPyPI.

### 4. Other Considerations
Since users may have different local environments from the one developments are based on, we are researching into methods that can help users install the dependencies of smartdiff.
We found two ways to handle dependency requirements: 

1. install_require() function from the setuptools package.

2. specify our own requirement file.

Basic idea of SmartDiff:

From the user input in GUI, we can simultaneously

1. render a latex expression of the function

2. get a python expression  

    For each component of the vector function, we divide it into chunks by parentheses (priority) and operators, then start from the innermost chunk(s) and evaluate the derivatives outward.

# Implementation

### 1. Core Data Structures
One of the core data structures is stack.  
When users type in the equations from left to right, they may create a few nested expressions. So the input parser needs to keep track of the terms at each level. Whenever we encounter an open parenthesis, we need to start a new level on top of the stack and add expression at that nested level as operand tuples.
Whenever a level terminates (i.e. the closing parenthesis is encountered), we need to pop the expression off from the top of stack and evaluate its derivative with respect to the target variable. 
If a terminated level of expression does not contain the target variable, we can simply leave it as a generic operand as a whole and continue the parsing.    

### 2. Classes, Methods and Name Attributes
For now, we have identified 5 basic classes as follows. However, we may add or delete some of them depending on how the fundamental algorithm works.
- Operand() class  
   - type: "numeral" or "var" since the type of each operand can either be a variable or a numerical value. If it is a mixed type, as long as there is a variable, its type will be "var".
   - value: string representation of the variable or numerical value
   - has_target_var: True if this operand contains the target variable (since this operand will be treated as a variable in the outer expression levels); False otherwise.
  
- Operator() class  
This is an Enum class that holds the string ID of each mathematical operators the differentiator can handle. Its attributes are the uppercase name of each operator.
  
- ElementaryGrad() class  
This class has a method for each type of operator above. Each method returns an new Operand object which is the derivative with respect to one input operand.
The base case of evaluating the derivative is 1. taking derivative of a single operand 2. taking derivative of two operands connected with an operator with respect to one operand.  
  
- InputScanner() class   
This class provides two methods described below:
   - tokenize(input_str, dx_str): scans the input string and outputs a nested list of operand-operator tuples and an operand for "dx_str".  
   - to_latex_format(input_str, dx_str): converts the input string into a latex string to be displayed in the GUI.
  
- Differentiator() class  
This class takes evaluates the derivative of a tokenized expression.
   - grad(expr, dx): It processes a nested list of tuples and iteratively computes the derivative with respect to the operand "dx".
   
- GUI() class  
This class will be based on the python GUI package, PyQt5 and uses the following predefined python classes to 
inform the user with instructions and error messages, direct the user to input the information, and display the output:
   - QComboBox, QLineEdit, QPushButton, QLabel, QCheckBox, QMessageBox


### 3. External Dependencies

   - PyQt5, for the UI
   - NumPy, for the solvers
   - Pandas, for generating the trace table
   - Pydoc, for viewing the function docstring
   - doctest, for unit testing of all the cases a function deals with (in a function docstring)
   - pytest, for testing files in test (extensive testing of boundaries between different modules)
   - setuptools & wheel, for packaging and distributing the code 
   - twine, for uploading the distribution archive files

### 4. Handle Elementary Functions like sin, sqrt, log, and exp (and all the others)

   We will be using operator overloading to define the derivative of each elementary functions in SmartDiff/solvers. For example, $\sin(x)$ will be mapped to $\cos(x)$ and $x+y$ will still be $x+y$.